In [13]:
from data import import_data as imp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [19]:
#Download data
# imp.import_raw_data()


raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'

input_path = raw_data_path + 'train.csv'
raw_data = pd.read_csv(input_path)

main_cols = ['SalePrice','Neighborhood','OverallQual','OverallCond','YearBuilt','YearRemodAdd','GrLivArea',
            'TotalBsmtSF','TotRmsAbvGrd','FullBath','GarageArea','BedroomAbvGr']

pre_data = raw_data[main_cols]

#Outlier removal
pre_data[pre_data.TotalBsmtSF > 5000]
pre_data.drop([1298],inplace = True)
pre_data.reset_index(drop=True,inplace=True)

pre_data[pre_data.GrLivArea > 4000]
pre_data.drop([523],inplace = True)
pre_data.reset_index(drop=True,inplace=True)

pre_data.to_csv(,index=False)

In [20]:
processed_data = pd.read_csv(processed_data_path + 'train_processed.csv')
processed_data

,SalePrice,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,GrLivArea,TotalBsmtSF,TotRmsAbvGrd,FullBath,GarageArea,BedroomAbvGr
0,208500,CollgCr,7,5,2003,2003,1710,856,8,2,548,3
1,181500,Veenker,6,8,1976,1976,1262,1262,6,2,460,3
2,223500,CollgCr,7,5,2001,2002,1786,920,6,2,608,3
3,140000,Crawfor,7,5,1915,1970,1717,756,7,1,642,3
4,250000,NoRidge,8,5,2000,2000,2198,1145,9,2,836,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1453,175000,Gilbert,6,5,1999,2000,1647,953,7,2,460,3
1454,210000,NWAmes,6,6,1978,1988,2073,1542,7,2,500,3
1455,266500,Crawfor,7,9,1941,2006,2340,1152,9,2,252,4
1456,142125,NAmes,5,6,1950,1996,1078,1078,5,1,240,2


In [21]:
from sklearn_pandas import FunctionTransformer
from sklearn_pandas import DataFrameMapper
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.compose import TransformedTargetRegressor


class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)
    
def pipeline_create():
    
    cols = ['Neighborhood', 'OverallQual', 'GrLivArea', 'YearBuilt']
    col_transformer = ColumnTransformer(
                    transformers=[
                        ('mms', MinMaxScaler(), ['YearBuilt']),
                        ('ohe', OneHotEncoder(handle_unknown="ignore"), ['Neighborhood'])             
                    ],
                    remainder='passthrough',
                    n_jobs=-1
                    )

    #Choose the ML model
    lr = LinearRegression()

    
    pipeline = Pipeline([('col_select',ColumnSelector(columns=cols)),
                        ("preprocessing", col_transformer),
                        ("lr", lr)])
     
    tt = TransformedTargetRegressor(regressor=pipeline, func=np.log, inverse_func=np.exp)
    return tt

def pipeline_create_fit(X,y):
    pipeline = pipeline_create()
    pipeline.fit(X,y)
    return pipeline

In [23]:
feature_data = processed_data.copy()

X_train, X_test, y_train, y_test = train_test_split(feature_data.drop('SalePrice',axis=1),
                                    feature_data.SalePrice, test_size=0.3, random_state=0)

pipeline = pipeline_create_fit(X_train,y_train)

train 0.025567712720538652
test 0.022301820703550735


In [40]:
from joblib import dump, load
from sklearn.metrics import mean_squared_log_error

def store_model_pipeline(file,path):
    dump(file, path + 'pipeline.joblib')

def load_model_pipeline(path):
    model = load(path) 
    return model

def model_predict(model,X):
    return np.round(model.predict(X),2)

def loss(y_pred, real):
    return mean_squared_log_error(y_pred,real)

In [41]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print('train',loss(train_pred,y_train))
print('test',loss(test_pred,y_test))

train 0.025567712720538652
test 0.022301820703550735


In [42]:
model_predict(model,X_train.iloc[[0]])

array([130363.35])

In [43]:
X_train.iloc[[0]]

,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,GrLivArea,TotalBsmtSF,TotRmsAbvGrd,FullBath,GarageArea,BedroomAbvGr
464,CollgCr,5,5,1978,1978,1040,1040,5,2,0,3
